# Content-based recommenders

Content-based recommenders in their recommendations rely purely on the features of items. Conceptually it can be expressed as a model of the form (personalized):
<center>
$$
    score \sim (user, item\_feature_1, item\_feature_2, ..., item\_feature_n)
$$
</center>
or (not personalized)
<center>
$$
    score \sim (item\_feature_1, item\_feature_2, ..., item\_feature_n)
$$
</center>

    + Content-based recommenders do not suffer from the cold-start problem for new items.
    - They do not use information about complex patterns of user-item interactions - what other similar users have already discovered and liked.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict
from sklearn.model_selection import KFold

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the data

In [2]:
ml_ratings_df = pd.read_csv(os.path.join("data", "movielens_small", "ratings.csv")).rename(columns={'userId': 'user_id', 'movieId': 'item_id'})
ml_movies_df = pd.read_csv(os.path.join("data", "movielens_small", "movies.csv")).rename(columns={'movieId': 'item_id'})
ml_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')
ml_df.head(10)

display(HTML(ml_movies_df.head(10).to_html()))

# Filter the data to reduce the number of movies
rng = np.random.RandomState(seed=6789)
left_ids = rng.choice(ml_movies_df['item_id'], size=1000, replace=False)

ml_ratings_df = ml_ratings_df.loc[ml_ratings_df['item_id'].isin(left_ids)]
ml_movies_df = ml_movies_df.loc[ml_movies_df['item_id'].isin(left_ids)]
ml_df = ml_df.loc[ml_df['item_id'].isin(left_ids)]

print("Number of left interactions: {}".format(len(ml_ratings_df)))

,item_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Number of left interactions: 9692


# Recommender class

Remark: Docstrings written in reStructuredText (reST) used by Sphinx to automatically generate code documentation. It is also used by default by PyCharm (type triple quotes after defining a class or a method and hit enter).

In [3]:
class Recommender(object):
    """
    Base recommender class.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        pass
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        pass
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            user_recommendations = pd.DataFrame({'user_id': user['user_id'],
                                                 'item_id': [-1] * n_recommendations,
                                                 'score': [3.0] * n_recommendations})

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations

# Evaluation measures

## Explicit feedback - ratings

### RMSE - Root Mean Squared Error

<center>
$$
    RMSE = \sqrt{\frac{\sum_{i}^N (\hat{r}_i - r_i)^2}{N}}
$$
</center>

where $\hat{r}_i$ are the predicted ratings and $r_i$ are the real ratings and $N$ is the number of items in the test set.

    + Very well-behaved analytically and therefore extensively used to train models, especially neural networks.
    - The scale of errors dependent on data which reduced comparability between different datasets.

In [4]:
def rmse(r_pred, r_real):
    return np.sqrt(np.sum(np.power(r_pred - r_real, 2)) / len(r_pred))

# Test

print("RMSE = {:.2f}".format(rmse(np.array([2.1, 1.2, 3.8, 4.2, 3.6]), np.array([3, 2, 4, 5, 1]))))

RMSE = 1.33


### MRE - Mean Relative Error

<center>
$$
    MRE = \frac{1}{N} \sum_{i}^N \frac{|\hat{r}_i - r_i|}{|r_i|}
$$
</center>

where $\hat{r}_i$ are the predicted ratings and $r_i$ are the real ratings and $N$ is the number of items in the test set.

    + Easily interpretable (average percentage error) and with a meaning understandable for business.
    - Blows up when there are values close to zero among the predicted values.

In [5]:
def mre(r_pred, r_real):
    return 1 / len(r_pred) * np.sum(np.abs(r_pred - r_real) / np.abs(r_real))

# Test

print("MRE = {:.4f}".format(mre(np.array([2.1, 1.2, 3.8, 4.2, 3.6]), np.array([3, 2, 4, 5, 1]))))

MRE = 0.7020


### TRE - Total Relative Error

<center>
$$
    TRE = \frac{\sum_{i}^N |\hat{r}_i - r_i|}{\sum_{i}^N |r_i|}
$$
</center>

where $\hat{r}_i$ are the predicted ratings and $r_i$ are the real ratings and $N$ is the number of items in the test set.

    + Easily interpretable (total percentage error) and with a meaning understandable for business.
    + Reliable even for very small predicted values.
    - Does not distinguish between a case when one prediction is very bad and other are very good and a case when all predictions are mediocre.

In [6]:
def tre(r_pred, r_real):
    return np.sum(np.abs(r_pred - r_real)) / np.sum(np.abs(r_real))

# Test

print("TRE = {:.4f}".format(tre(np.array([2.1, 1.2, 3.8, 4.2, 3.6]), np.array([3, 2, 4, 5, 1]))))

TRE = 0.3533


## Implicit feedback - binary indicators of interactions

### HR@n - Hit Ratio 
How many hits did we score in the first n recommendations.
<br/>
<br/>
<center>
$$
    \text{HR@}n = \frac{\sum_{u} \sum_{i \in I_u} r_{u, i} \cdot 1_{\hat{D}_n(u)}(i)}{M}
$$
</center>

where:
  * $r_{u, i}$ is $1$ if there was an interaction between user $u$ and item $i$ in the test set and $0$ otherwise, 
  * $\hat{D}_n$ is the set of the first $n$ recommendations for user $u$, 
  * $1_{\hat{D}_n}(i)$ is $1$ if and only if $i \in \hat{D}_n$, otherwise it's equal to $0$,
  * $M$ is the number of users.


    + Easily interpretable.
    - Does not take the rank of each recommendation into account.

In [7]:
def hr(recommendations, real_interactions, n=1):
    """
    Assumes recommendations are ordered by user_id and then by score.
    """
    # Transform real_interactions to a dict for a large speed-up
    rui = defaultdict(lambda: 0)
    
    for idx, row in real_interactions.iterrows():
        rui[(row['user_id'], row['item_id'])] = 1
        
    hr = 0.0
    
    previous_user_id = -1
    rank = 0
    for idx, row in recommendations.iterrows():
        if previous_user_id == row['user_id']:
            rank += 1
        else:
            rank = 1
            
        if rank <= n:
            hr += rui[(row['user_id'], row['item_id'])]
        
        previous_user_id = row['user_id']
    
    hr /= len(recommendations['user_id'].unique())
    
    return hr

    
recommendations = pd.DataFrame(
    [
        [1, 13, 0.9],
        [1, 45, 0.8],
        [1, 22, 0.71],
        [1, 77, 0.55],
        [1, 9, 0.52],
        [2, 11, 0.85],
        [2, 13, 0.69],
        [2, 25, 0.64],
        [2, 6, 0.60],
        [2, 77, 0.53]
        
    ], columns=['user_id', 'item_id', 'score'])

display(HTML(recommendations.to_html()))

real_interactions = pd.DataFrame(
    [
        [1, 45],
        [1, 22],
        [1, 77],
        [2, 13],
        [2, 77]
        
    ], columns=['user_id', 'item_id'])

display(HTML(real_interactions.to_html()))
    
print("HR@3 = {:.4f}".format(hr(recommendations, real_interactions, n=3)))

,user_id,item_id,score
0,1,13,0.90
1,1,45,0.80
2,1,22,0.71
3,1,77,0.55
4,1,9,0.52
5,2,11,0.85
6,2,13,0.69
7,2,25,0.64
8,2,6,0.60
9,2,77,0.53


,user_id,item_id
0,1,45
1,1,22
2,1,77
3,2,13
4,2,77


HR@3 = 1.5000


### NDCG@n - Normalized Discounted Cumulative Gain

How many hits did we score in the first n recommendations discounted by the position of each recommendation.
<br/>
<br/>
<center>
$$
    \text{NDCG@}n = \frac{\sum_{u} \sum_{i \in I_u} \frac{r_{u, i}}{log\left(1 + v_{\hat{D}_n(u)}(i)\right)}}{M}
$$
</center>

where:
  * $r_{u, i}$ is $1$ if there was an interaction between user $u$ and item $i$ in the test set and $0$ otherwise, 
  * $\hat{D}_n(u)$ is the set of the first $n$ recommendations for user $u$, 
  * $v_{\hat{D}_n(u)}(i)$ is the position of item $i$ in recommendations $\hat{D}_n$,
  * $M$ is the number of users.


    - Takes the rank of each recommendation into account.

In [8]:
def ndcg(recommendations, real_interactions, n=1):
    """
    Assumes recommendations are ordered by user_id and then by score.
    """
    # Transform real_interactions to a dict for a large speed-up
    rui = defaultdict(lambda: 0)
    
    for idx, row in real_interactions.iterrows():
        rui[(row['user_id'], row['item_id'])] = 1
        
    ndcg = 0.0
    
    previous_user_id = -1
    rank = 0
    for idx, row in recommendations.iterrows():
        if previous_user_id == row['user_id']:
            rank += 1
        else:
            rank = 1
            
        if rank <= n:
            ndcg += rui[(row['user_id'], row['item_id'])] / np.log2(1 + rank)
        
        previous_user_id = row['user_id']
    
    ndcg /= len(recommendations['user_id'].unique())
    
    return ndcg

    
recommendations = pd.DataFrame(
    [
        [1, 13, 0.9],
        [1, 45, 0.8],
        [1, 22, 0.71],
        [1, 77, 0.55],
        [1, 9, 0.52],
        [2, 11, 0.85],
        [2, 13, 0.69],
        [2, 25, 0.64],
        [2, 6, 0.60],
        [2, 77, 0.53]
        
    ], columns=['user_id', 'item_id', 'score'])

display(HTML(recommendations.to_html()))

real_interactions = pd.DataFrame(
    [
        [1, 45],
        [1, 22],
        [1, 77],
        [2, 13],
        [2, 77]
        
    ], columns=['user_id', 'item_id'])

display(HTML(real_interactions.to_html()))
    
print("NDCG@3 = {:.4f}".format(ndcg(recommendations, real_interactions, n=3)))

,user_id,item_id,score
0,1,13,0.90
1,1,45,0.80
2,1,22,0.71
3,1,77,0.55
4,1,9,0.52
5,2,11,0.85
6,2,13,0.69
7,2,25,0.64
8,2,6,0.60
9,2,77,0.53


,user_id,item_id
0,1,45
1,1,22
2,1,77
3,2,13
4,2,77


NDCG@3 = 0.8809


# Testing routines (offline)

## Train and test set split

### Explicit feedback

In [9]:
def evaluate_train_test_split_explicit(recommender, interactions_df, items_df, seed=6789):
    rng = np.random.RandomState(seed=seed)
    
    # Split the dataset into train and test
    
    shuffle = np.arange(len(interactions_df))
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    interactions_df_train = interactions_df.iloc[shuffle[:split_index]]
    interactions_df_test = interactions_df.iloc[shuffle[split_index:]]
    
    # Train the recommender
    
    recommender.fit(interactions_df_train, None, items_df)
    
    # Gather predictions
    
    r_pred = []
    
    for idx, row in interactions_df_test.iterrows():
        users_df = pd.DataFrame([row['user_id']], columns=['user_id'])
        eval_items_df = pd.DataFrame([row['item_id']], columns=['item_id'])
        print(eval_items_df)
        eval_items_df = pd.merge(eval_items_df, items_df, on='item_id')
        print(eval_items_df)
        recommendations = recommender.recommend(users_df, eval_items_df, n_recommendations=1)
        
        r_pred.append(recommendations.iloc[0]['score'])
    
    # Gather real ratings
    
    r_real = np.array(interactions_df_test['rating'].tolist())
    # Return evaluation metrics
    
    return rmse(r_pred, r_real), mre(r_pred, r_real), tre(r_pred, r_real)

recommender = Recommender()

results = [['BaseRecommender'] + list(evaluate_train_test_split_explicit(
    recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(HTML(results.to_html()))

   item_id
0   3751.0
   item_id               title                     genres
0   3751.0  Chicken Run (2000)  Animation|Children|Comedy
   item_id
0   7438.0
   item_id                     title                 genres
0   7438.0  Kill Bill: Vol. 2 (2004)  Action|Drama|Thriller
   item_id
0  91842.0
   item_id              title                       genres
0  91842.0  Contraband (2012)  Action|Crime|Drama|Thriller
   item_id
0   1500.0
   item_id                       title                genres
0   1500.0  Grosse Pointe Blank (1997)  Comedy|Crime|Romance
   item_id
0   1955.0
   item_id                     title genres
0   1955.0  Kramer vs. Kramer (1979)  Drama
   item_id
0   7438.0
   item_id                     title                 genres
0   7438.0  Kill Bill: Vol. 2 (2004)  Action|Drama|Thriller
   item_id
0   3639.0
   item_id                                title                     genres
0   3639.0  Man with the Golden Gun, The (1974)  Action|Adventure|Thriller
   item_id
0

0    930.0
   item_id             title                      genres
0    930.0  Notorious (1946)  Film-Noir|Romance|Thriller
   item_id
0    842.0
   item_id                                              title         genres
0    842.0  Tales from the Crypt Presents: Bordello of Blo...  Comedy|Horror
   item_id
0   2790.0
   item_id                                              title           genres
0   2790.0  Final Conflict, The (a.k.a. Omen III: The Fina...  Horror|Thriller
   item_id
0   7438.0
   item_id                     title                 genres
0   7438.0  Kill Bill: Vol. 2 (2004)  Action|Drama|Thriller
   item_id
0  68358.0
   item_id             title                        genres
0  68358.0  Star Trek (2009)  Action|Adventure|Sci-Fi|IMAX
   item_id
0   2311.0
   item_id                                  title  genres
0   2311.0  2010: The Year We Make Contact (1984)  Sci-Fi
   item_id
0    595.0
   item_id                        title  \
0    595.0  Beauty and the Beast (

0  71899.0
   item_id                title                  genres
0  71899.0  Mary and Max (2009)  Animation|Comedy|Drama
   item_id
0     50.0
   item_id                       title                  genres
0     50.0  Usual Suspects, The (1995)  Crime|Mystery|Thriller
   item_id
0   1225.0
   item_id           title genres
0   1225.0  Amadeus (1984)  Drama
   item_id
0    423.0
   item_id              title           genres
0    423.0  Blown Away (1994)  Action|Thriller
    item_id
0  180031.0
    item_id                      title                   genres
0  180031.0  The Shape of Water (2017)  Adventure|Drama|Fantasy
   item_id
0   4255.0
   item_id                       title  genres
0   4255.0  Freddy Got Fingered (2001)  Comedy
   item_id
0   6688.0
   item_id                             title        genres
0   6688.0  Autumn Spring (Babí léto) (2001)  Comedy|Drama
   item_id
0  67295.0
   item_id                                              title  \
0  67295.0  Kung Fu Panda: S

0   3536.0
   item_id                     title                genres
0   3536.0  Keeping the Faith (2000)  Comedy|Drama|Romance
   item_id
0   2952.0
   item_id                       title                genres
0   2952.0  Sydney (Hard Eight) (1996)  Crime|Drama|Thriller
   item_id
0    379.0
   item_id           title                  genres
0    379.0  Timecop (1994)  Action|Sci-Fi|Thriller
   item_id
0   2716.0
   item_id                                       title                genres
0   2716.0  Ghostbusters (a.k.a. Ghost Busters) (1984)  Action|Comedy|Sci-Fi
   item_id
0    233.0
   item_id           title genres
0    233.0  Exotica (1994)  Drama
   item_id
0   1976.0
   item_id                              title  genres
0   1976.0  Friday the 13th Part 3: 3D (1982)  Horror
   item_id
0   7438.0
   item_id                     title                 genres
0   7438.0  Kill Bill: Vol. 2 (2004)  Action|Drama|Thriller
   item_id
0   2065.0
   item_id                             titl

   item_id                                 title  \
0    780.0  Independence Day (a.k.a. ID4) (1996)   

                             genres  
0  Action|Adventure|Sci-Fi|Thriller  
   item_id
0    858.0
   item_id                  title       genres
0    858.0  Godfather, The (1972)  Crime|Drama
   item_id
0    959.0
   item_id                    title genres
0    959.0  Of Human Bondage (1934)  Drama
   item_id
0   6502.0
   item_id                 title                genres
0   6502.0  28 Days Later (2002)  Action|Horror|Sci-Fi
   item_id
0   4896.0
   item_id                                              title  \
0   4896.0  Harry Potter and the Sorcerer's Stone (a.k.a. ...   

                       genres  
0  Adventure|Children|Fantasy  
   item_id
0   6502.0
   item_id                 title                genres
0   6502.0  28 Days Later (2002)  Action|Horror|Sci-Fi
   item_id
0    585.0
   item_id                          title  genres
0    585.0  Brady Bunch Movie, The (1995) 

...

0    266.0
   item_id                       title                     genres
0    266.0  Legends of the Fall (1994)  Drama|Romance|War|Western
    item_id
0  133545.0
    item_id                    title        genres
0  133545.0  Just Before I Go (2014)  Comedy|Drama
   item_id
0   1107.0
   item_id         title  genres
0   1107.0  Loser (1991)  Comedy
   item_id
0  69436.0
   item_id            title            genres
0  69436.0  Year One (2009)  Adventure|Comedy
   item_id
0    788.0
   item_id                        title                         genres
0    788.0  Nutty Professor, The (1996)  Comedy|Fantasy|Romance|Sci-Fi
   item_id
0   5040.0
   item_id                       title                    genres
0   5040.0  Conan the Destroyer (1984)  Action|Adventure|Fantasy
   item_id
0   3671.0
   item_id                   title          genres
0   3671.0  Blazing Saddles (1974)  Comedy|Western
   item_id
0   5464.0
   item_id                     title       genres
0   5464.0  Road 

0   3213.0
   item_id                                title              genres
0   3213.0  Batman: Mask of the Phantasm (1993)  Animation|Children
   item_id
0   6537.0
   item_id                                      title                   genres
0   6537.0  Terminator 3: Rise of the Machines (2003)  Action|Adventure|Sci-Fi
   item_id
0   2420.0
   item_id                   title genres
0   2420.0  Karate Kid, The (1984)  Drama
   item_id
0  60684.0
   item_id            title                                     genres
0  60684.0  Watchmen (2009)  Action|Drama|Mystery|Sci-Fi|Thriller|IMAX
   item_id
0  69844.0
   item_id                                          title  \
0  69844.0  Harry Potter and the Half-Blood Prince (2009)   

                                   genres  
0  Adventure|Fantasy|Mystery|Romance|IMAX  
   item_id
0    481.0
   item_id              title          genres
0    481.0  Kalifornia (1993)  Drama|Thriller
   item_id
0  39292.0
   item_id                        

   item_id                   title                                  genres
0  48774.0  Children of Men (2006)  Action|Adventure|Drama|Sci-Fi|Thriller
   item_id
0    543.0
   item_id                                title                   genres
0    543.0  So I Married an Axe Murderer (1993)  Comedy|Romance|Thriller
    item_id
0  126548.0
    item_id            title  genres
0  126548.0  The DUFF (2015)  Comedy
   item_id
0    858.0
   item_id                  title       genres
0    858.0  Godfather, The (1972)  Crime|Drama
   item_id
0   2313.0
   item_id                     title genres
0   2313.0  Elephant Man, The (1980)  Drama
   item_id
0   7482.0
   item_id                    title        genres
0   7482.0  Enter the Dragon (1973)  Action|Crime
   item_id
0   5103.0
   item_id                title                 genres
0   5103.0  Sandlot, The (1993)  Children|Comedy|Drama
   item_id
0   8798.0
   item_id              title                       genres
0   8798.0  Collateral 

   item_id        title       genres
0   4239.0  Blow (2001)  Crime|Drama
   item_id
0   1704.0
   item_id                     title         genres
0   1704.0  Good Will Hunting (1997)  Drama|Romance
   item_id
0   4896.0
   item_id                                              title  \
0   4896.0  Harry Potter and the Sorcerer's Stone (a.k.a. ...   

                       genres  
0  Adventure|Children|Fantasy  
   item_id
0    930.0
   item_id             title                      genres
0    930.0  Notorious (1946)  Film-Noir|Romance|Thriller
   item_id
0   2010.0
   item_id              title        genres
0   2010.0  Metropolis (1927)  Drama|Sci-Fi
   item_id
0   3635.0
   item_id                         title                     genres
0   3635.0  Spy Who Loved Me, The (1977)  Action|Adventure|Thriller
   item_id
0    274.0
   item_id                    title  genres
0    274.0  Man of the House (1995)  Comedy
    item_id
0  111146.0
    item_id                                  

,Recommender,RMSE,MRE,TRE
0,BaseRecommender,1.170155,0.349264,0.271796


### Implicit feedback

**Task 1.** Implement the following method for train-test split evaluation for implicit feedback.

In [12]:
def evaluate_train_test_split_implicit(recommender, interactions_df, items_df, seed=6789):
    rng = np.random.RandomState(seed=seed)
    # Split the dataset into train and test
    shuffle = np.arange(len(interactions_df))
    rng.shuffle(shuffle)
    shuffle = list(shuffle)
    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)
    interactions_df_train = interactions_df.iloc[shuffle[:split_index]]
    interactions_df_test = interactions_df.iloc[shuffle[split_index:]]
    # Train the recommender
    
    recommender.fit(interactions_df_train, None, items_df)
    
    
    hr_1 = []
    hr_3 = []
    hr_5 = []
    hr_10 = []
    
    users_ids = interactions_df_test.loc[:, ['user_id']].drop_duplicates(subset='user_id')
    recommendations = recommender.recommend(users_ids, items_df, n_recommendations=10)
#     display(HTML(recommendations.to_html()))
    hr_1.append(hr(recommendations, interactions_df_test, n=1))
    hr_3.append(hr(recommendations, interactions_df_test, n=3))
    hr_5.append(hr(recommendations, interactions_df_test, n=5))
    hr_10.append(hr(recommendations, interactions_df_test, n=10))
    
    hr_1 = np.mean(hr_1)
    hr_3 = np.mean(hr_3)
    hr_5 = np.mean(hr_5)
    hr_10 = np.mean(hr_10)
    
    return hr_1, hr_3, hr_5, hr_10

recommender = TFIDFRecommender()

results = [['BaseRecommender'] + list(evaluate_train_test_split_implicit(
    recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10'])

display(HTML(results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10
0,BaseRecommender,0.017897,0.044743,0.058166,0.09396


## Leave-one-out, leave-k-out, cross-validation

### Explicit feedback

**Task 2.** Implement the following method for leave-one-out evaluation for explicit feedback.

In [15]:
def evaluate_leave_one_out_explicit(recommender, interactions_df, items_df, max_evals=300, seed=6789):
    rng = np.random.RandomState(seed=seed)
    
    # Prepare splits of the datasets
    kf = KFold(n_splits=len(interactions_df), random_state=rng, shuffle=True)
    

    # For each split of the dataset train the recommender, generate recommendations and evaluate
    r_pred = []
    r_real = []
    n_eval = 1
    for train_index, test_index in kf.split(interactions_df.index):
        interactions_df_train = interactions_df.loc[interactions_df.index[train_index]]
        interactions_df_test = interactions_df.loc[interactions_df.index[test_index]]
                
        recommender.fit(interactions_df_train, None, items_df)
        item = interactions_df_test.loc[:, ['item_id']]
        item = pd.merge(item, items_df, on='item_id')
        recommendations = recommender.recommend(interactions_df_test.loc[:, ['user_id']], 
                                                item, 
                                                n_recommendations=1)
        
        r_pred.append(recommendations.iloc[0]['score'])
        r_real.append(interactions_df_test['rating'].tolist()[0])
        if n_eval == max_evals:
            break
        n_eval += 1
    
    r_real = np.array(r_real)
#     print(r_pred)
#     print(r_real)
    return rmse(r_pred, r_real), mre(r_pred, r_real), tre(r_pred, r_real)

recommender = LinearRegressionRecommender()
results = [['BaseRecommender'] + list(evaluate_leave_one_out_explicit(
    recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df))]
results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])
display(HTML(results.to_html()))

,Recommender,RMSE,MRE,TRE
0,BaseRecommender,1.039763,0.401878,0.240408


### Implicit feedback

# Linear Regression Recommender

For every movie we transform its genres into one-hot encoded features and then fit a linear regression model to those features and actual ratings.

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MultiLabelBinarizer

class LinearRegressionRecommender(object):
    """
    Base recommender class.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        self.model = None
        self.mlb = None
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        interactions_df = pd.merge(interactions_df, items_df, on='item_id')
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("-", "_", regex=False)
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(" ", "_", regex=False)
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.lower()
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.split("|")
        
        self.mlb = MultiLabelBinarizer()
        interactions_df = interactions_df.join(
            pd.DataFrame(self.mlb.fit_transform(interactions_df.pop('genres')),
                         columns=self.mlb.classes_,
                         index=interactions_df.index))
        
#         print(interactions_df.head())
        
        x = interactions_df.loc[:, self.mlb.classes_].values
        y = interactions_df['rating'].values
    
        self.model = LinearRegression().fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Transform the item to be scored into proper features
        
        items_df = items_df.copy()
        items_df.loc[:, 'genres'] = items_df['genres'].str.replace("-", "_", regex=False)
        items_df.loc[:, 'genres'] = items_df['genres'].str.replace(" ", "_", regex=False)
        items_df.loc[:, 'genres'] = items_df['genres'].str.lower()
        items_df.loc[:, 'genres'] = items_df['genres'].str.split("|")
        
        items_df = items_df.join(
            pd.DataFrame(self.mlb.transform(items_df.pop('genres')),
                         columns=self.mlb.classes_,
                         index=items_df.index))
        
#         print(items_df)
        
        # Score the item
    
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            score = self.model.predict(items_df.loc[:, self.mlb.classes_].values)[0]
                
            user_recommendations = pd.DataFrame({'user_id': [user['user_id']],
                                                 'item_id': items_df.iloc[0]['item_id'],
                                                 'score': score})

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations

In [14]:
# Quick test of the recommender

lr_recommender = LinearRegressionRecommender()
lr_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = lr_recommender.recommend(pd.DataFrame([[1], [2]], columns=['user_id']), ml_movies_df, 1)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id')
display(HTML(recommendations.to_html()))

,user_id,item_id,score,title,genres
0,1,145,3.282778,Bad Boys (1995),Action|Comedy|Crime|Drama|Thriller
1,2,145,3.282778,Bad Boys (1995),Action|Comedy|Crime|Drama|Thriller


In [15]:
lr_recommender = LinearRegressionRecommender()

results = [['LinearRegressionRecommender'] + list(evaluate_train_test_split_explicit(
    lr_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, seed=6789))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(HTML(results.to_html()))

,Recommender,RMSE,MRE,TRE
0,LinearRegressionRecommender,1.019263,0.370077,0.245447


# TF-IDF Recommender
TF-IDF stands for term frequency–inverse document frequency. Typically Tf-IDF method is used to assign keywords (words describing the gist of a document) to documents in a corpus of documents.

In our case we will treat users as documents and genres as words.

Term-frequency is given by the following formula:
<center>
$$
    \text{tf}(g, u) = f_{g, u}
$$
</center>
where $f_{g, i}$ is the number of times genre $g$ appear for movies watched by user $u$.

Inverse document frequency is defined as follows:
<center>
$$
    \text{idf}(g) = \log \frac{N}{n_g}
$$
</center>
where $N$ is the number of users and $n_g$ is the number of users with $g$ in their genres list.

Finally, tf-idf is defined as follows:
<center>
$$
    \text{tfidf}(g, u) = \text{tf}(g, u) \cdot \text{idf}(g)
$$
</center>

In our case we will measure how often a given genre appears for movies watched by a given user vs how often it appears for all users. To obtain a movie score we will take the average of its genres' scores for this user.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

class TFIDFRecommender(object):
    """
    Recommender based on the TF-IDF method.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        self.tfidf_scores = None
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        self.tfidf_scores = defaultdict(lambda: 0.0)

        # Prepare the corpus for tfidf calculation
        
        interactions_df = pd.merge(interactions_df, items_df, on='item_id')
        user_genres = interactions_df.loc[:, ['user_id', 'genres']]
        user_genres.loc[:, 'genres'] = user_genres['genres'].str.replace("-", "_", regex=False)
        user_genres.loc[:, 'genres'] = user_genres['genres'].str.replace(" ", "_", regex=False)
        user_genres = user_genres.groupby('user_id').aggregate(lambda x: "|".join(x))
        user_genres.loc[:, 'genres'] = user_genres['genres'].str.replace("|", " ", regex=False)
#         print(user_genres)
        user_ids = user_genres.index.tolist()
        genres_corpus = user_genres['genres'].tolist()
        
        # Calculate tf-idf scores
        
        vectorizer = TfidfVectorizer()
        tfidf_scores = vectorizer.fit_transform(genres_corpus)
        
        # Transform results into a dict {(user_id, genre): score}
        
        for u in range(tfidf_scores.shape[0]):
            for g in range(tfidf_scores.shape[1]):
                self.tfidf_scores[(user_ids[u], vectorizer.get_feature_names()[g])] = tfidf_scores[u, g]
                
#         print(self.tfidf_scores)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        # Transform genres to a unified form used by the vectorizer
        
        items_df = items_df.copy()
        items_df.loc[:, 'genres'] = items_df['genres'].str.replace("-", "_", regex=False)
        items_df.loc[:, 'genres'] = items_df['genres'].str.replace(" ", "_", regex=False)
        items_df.loc[:, 'genres'] = items_df['genres'].str.lower()
        items_df.loc[:, 'genres'] = items_df['genres'].str.split("|")
                
        # Score items    
        
        for uix, user in users_df.iterrows():
            items = []
            for iix, item in items_df.iterrows():
                score = 0.0
                for genre in item['genres']:
                    score += self.tfidf_scores[(user['user_id'], genre)]
                score /= len(item['genres'])
                items.append((item['item_id'], score))
                
            items = sorted(items, key=lambda x: x[1], reverse=True)
            user_recommendations = pd.DataFrame({'user_id': user['user_id'],
                                                 'item_id': [item[0] for item in items][:n_recommendations],
                                                 'score': [item[1] for item in items][:n_recommendations]})

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations

In [17]:
# Quick test of the recommender

tfidf_recommender = TFIDFRecommender()
tfidf_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = tfidf_recommender.recommend(pd.DataFrame([[1], [2]], columns=['user_id']), ml_movies_df, 3)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id')
display(HTML(recommendations.to_html()))

,user_id,item_id,score,title,genres
0,1,2690,0.452122,"Ideal Husband, An (1999)",Comedy|Romance
1,1,3614,0.452122,Honeymoon in Vegas (1992),Comedy|Romance
2,1,4796,0.452122,"Grass Is Greener, The (1960)",Comedy|Romance
3,2,145,0.000000,Bad Boys (1995),Action|Comedy|Crime|Drama|Thriller
4,2,171,0.000000,Jeffrey (1995),Comedy|Drama
5,2,228,0.000000,Destiny Turns on the Radio (1995),Comedy


In [18]:
tfidf_recommender = TFIDFRecommender()

results = [['TFIDFRecommender'] + list(evaluate_leave_one_out_implicit(
    tfidf_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df, max_evals=300, seed=6789))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,TFIDFRecommender,0.006667,0.053333,0.123333,0.233333,0.006667,0.033491,0.062178,0.096151


In [16]:
def evaluate_leave_one_out_implicit(recommender, interactions_df, items_df, max_evals=300, seed=6789):
    rng = np.random.RandomState(seed=seed)
    
    # Prepare splits of the datasets
    kf = KFold(n_splits=len(interactions_df), random_state=rng, shuffle=True)
    
    hr_1 = []
    hr_3 = []
    hr_5 = []
    hr_10 = []
    ndcg_1 = []
    ndcg_3 = []
    ndcg_5 = []
    ndcg_10 = []
    
    # For each split of the dataset train the recommender, generate recommendations and evaluate
    
    n_eval = 1
    for train_index, test_index in kf.split(interactions_df.index):
        interactions_df_train = interactions_df.loc[interactions_df.index[train_index]]
        interactions_df_test = interactions_df.loc[interactions_df.index[test_index]]
                
        recommender.fit(interactions_df_train, None, items_df)
        recommendations = recommender.recommend(interactions_df_test.loc[:, ['user_id']], items_df, n_recommendations=10)
        
        hr_1.append(hr(recommendations, interactions_df_test, n=1))
        hr_3.append(hr(recommendations, interactions_df_test, n=3))
        hr_5.append(hr(recommendations, interactions_df_test, n=5))
        hr_10.append(hr(recommendations, interactions_df_test, n=10))
        ndcg_1.append(ndcg(recommendations, interactions_df_test, n=1))
        ndcg_3.append(ndcg(recommendations, interactions_df_test, n=3))
        ndcg_5.append(ndcg(recommendations, interactions_df_test, n=5))
        ndcg_10.append(ndcg(recommendations, interactions_df_test, n=10))
        
        if n_eval == max_evals:
            break
        n_eval += 1
        
    hr_1 = np.mean(hr_1)
    hr_3 = np.mean(hr_3)
    hr_5 = np.mean(hr_5)
    hr_10 = np.mean(hr_10)
    ndcg_1 = np.mean(ndcg_1)
    ndcg_3 = np.mean(ndcg_3)
    ndcg_5 = np.mean(ndcg_5)
    ndcg_10 = np.mean(ndcg_10)
    
    return hr_1, hr_3, hr_5, hr_10, ndcg_1, ndcg_3, ndcg_5, ndcg_10

# recommender = TFIDFRecommender()

# results = [['BaseRecommender'] + list(evaluate_leave_one_out_implicit(
#     recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df))]

# results = pd.DataFrame(results, 
#                        columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

# display(HTML(results.to_html()))

recommender = MostPopularRecommender()
results = [['MostPopularRecommender'] + list(evaluate_leave_one_out_implicit(
    recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,MostPopularRecommender,0.026667,0.05,0.083333,0.163333,0.026667,0.040079,0.053997,0.079749


# Tasks

**Task 3.** Implement the MostPopularRecommender (check the slides for class 1), evaluate it with leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [15]:
class MostPopularRecommender(object):
    """
    Most popular recommender class.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        self.most_popular = []
        pass
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        grouped = interactions_df.groupby('item_id').size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
#         grouped = interactions_df.groupby('item_id').count().sort_values(by=['user_id'], ascending=False)
        self.most_popular = list(grouped['item_id'])
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            user_recommendations = pd.DataFrame({'user_id': user['user_id'],
                                                 'item_id': self.most_popular[:n_recommendations],
                                                 'score': [3.0] * n_recommendations})

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations

recommender = MostPopularRecommender()
results = [['MostPopularRecommender'] + list(evaluate_leave_one_out_implicit(
    recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df, max_evals=300, seed=6789))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,MostPopularRecommender,0.026667,0.05,0.083333,0.163333,0.026667,0.040079,0.053997,0.079749


**Task 4.** Implement the HighestRatedRecommender (check the slides for class 1), evaluate it with leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [56]:
class HighestRatedRecommender(object):
    """
    Highest rated recommender class.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        self.highest_rated = []
        pass
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        grouped = interactions_df.drop(columns=['user_id']).groupby('item_id').mean('rating').sort_values(by=['rating', 'item_id'], ascending=[False, True])
        self.highest_rated = list(grouped.index)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            user_recommendations = pd.DataFrame({'user_id': user['user_id'],
                                                 'item_id': self.highest_rated[:n_recommendations],
                                                 'score': [3.0] * n_recommendations})

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations

recommender = HighestRatedRecommender()
results = [['HighestRatedRecommender'] + list(evaluate_leave_one_out_implicit(
    recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, max_evals=300, seed=6789))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,HighestRatedRecommender,0.0,0.0,0.0,0.003333,0.0,0.0,0.0,0.001111


**Task 5.** Implement the RandomRecommender (check the slides for class 1), evaluate it with leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [19]:
class RandomRecommender(object):
    """
    Random recommender class.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        self.shuffled = []
        self.rng = np.random.RandomState(seed=6789)
        pass
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        shuffle = np.arange(len(items_df))
        self.rng.shuffle(shuffle)
        shuffle = list(shuffle)
        
        self.shuffled = [items_df.iloc[x]['item_id'] for x in shuffle]
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            user_recommendations = pd.DataFrame({'user_id': user['user_id'],
                                                 'item_id': self.shuffled[:n_recommendations],
                                                 'score': [3.0] * n_recommendations})

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations

recommender = RandomRecommender()
recommender.fit(ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], None, ml_movies_df)
results = [['RandomRecommender'] + list(evaluate_leave_one_out_implicit(
    recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, max_evals=300, seed=6789))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomRecommender,0.0,0.003333,0.006667,0.006667,0.0,0.001667,0.003102,0.003102


**Task 6.** Gather the results for TFIDFRecommender, MostPopularRecommender, HighestRatedRecommender, RandomRecommender in one DataFrame and print it.

In [54]:
tfidf_recommender = TFIDFRecommender()
tf_results = [['TFIDFRecommender'] + list(evaluate_leave_one_out_implicit(
    tfidf_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df, max_evals=300, seed=6789))]

tf_results = pd.DataFrame(tf_results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])


recommender = MostPopularRecommender()
mp_results = [['MostPopularRecommender'] + list(evaluate_leave_one_out_implicit(
    recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df, max_evals=300, seed=6789))]

mp_results = pd.DataFrame(mp_results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])


recommender = HighestRatedRecommender()
hr_results = [['HighestRatedRecommender'] + list(evaluate_leave_one_out_implicit(
    recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, max_evals=300, seed=6789))]

hr_results = pd.DataFrame(hr_results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])


recommender = RandomRecommender()
r_results = [['RandomRecommender'] + list(evaluate_leave_one_out_implicit(
    recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, max_evals=300, seed=6789))]

r_results = pd.DataFrame(r_results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])


result = pd.concat([tf_results, mp_results, hr_results, r_results])
result

[472, 26631, 63276, 2815, 31553, 46335, 1107, 42728, 4558, 77866, 150254, 146662, 2713, 7256, 139717, 112552, 145418, 6751, 132454, 48982, 2739, 27865, 1937, 76738, 1449, 6688, 150993, 1130, 95313, 5155, 99, 96964, 5384, 6342, 27134, 44613, 2313, 70697, 71732, 7832, 168846, 3912, 1777, 3451, 146684, 2307, 207, 3053, 61406, 103606, 1079, 2068, 62437, 979, 2059, 1704, 87869, 82684, 67618, 149011, 2878, 3479, 114818, 7991, 4629, 8581, 69640, 26622, 606, 837, 182715, 3564, 3557, 2504, 7071, 1322, 2301, 4978, 4135, 77191, 2574, 327, 41617, 3687, 189381, 175485, 875, 4573, 68650, 27644, 3668, 6223, 81191, 3394, 1914, 168252, 168456, 4987, 179953, 3628, 842, 89678, 3165, 8861, 3017, 112556, 3468, 163925, 7833, 161830, 3000, 88267, 2046, 4006, 68536, 89386, 8331, 175431, 5636, 113159, 423, 3797, 55272, 3439, 5450, 50356, 44195, 171701, 5093, 3846, 4967, 4259, 1021, 4255, 7988, 151315, 32460, 6720, 7451, 8753, 5752, 1033, 5489, 3981, 1707, 3211, 143365, 1891, 5231, 59018, 8366, 132488, 5041, 21

[108795, 4773, 5513, 8340, 7988, 6927, 95004, 5585, 2088, 1489, 34520, 143472, 274, 3053, 4745, 102716, 7179, 279, 55444, 26265, 69640, 3017, 6342, 7646, 48774, 813, 1151, 31553, 5562, 173209, 8581, 116529, 6193, 8238, 6720, 39292, 70946, 82767, 4947, 3327, 4139, 460, 3667, 27772, 33124, 95873, 2935, 472, 5231, 3507, 3640, 2716, 2815, 56801, 134334, 6216, 1760, 117529, 477, 152105, 145, 313, 750, 60408, 1707, 64501, 2964, 2932, 40819, 70451, 3899, 159061, 101360, 7340, 212, 1862, 346, 4927, 1996, 208, 1107, 3067, 172577, 2661, 423, 103606, 2275, 6755, 790, 70697, 77191, 8908, 7832, 61628, 3211, 8506, 5455, 3021, 135861, 1223, 1353, 4488, 1859, 3394, 875, 959, 3313, 165103, 100390, 3684, 139717, 3846, 2682, 6549, 59018, 3479, 121372, 280, 6301, 158872, 4787, 6808, 6713, 95508, 112818, 80126, 5464, 59604, 74510, 90746, 5212, 95313, 2295, 82167, 2671, 5668, 59336, 3219, 26357, 131578, 6390, 68536, 131656, 122892, 2690, 6039, 5710, 1040, 6884, 3341, 151763, 156553, 32799, 41573, 233, 6581,

[40966, 343, 31921, 4115, 7071, 25962, 280, 1893, 1963, 32294, 26133, 2082, 139747, 180, 1223, 4002, 56339, 103606, 99, 134334, 95147, 3768, 112818, 4616, 3799, 175485, 6187, 146662, 3591, 95165, 2594, 952, 37741, 173941, 2964, 64575, 6927, 134021, 60684, 53000, 5633, 2713, 8605, 994, 94867, 31909, 30822, 8987, 1172, 132796, 2517, 3330, 6058, 7714, 163925, 7116, 5954, 31867, 1439, 3640, 379, 3684, 93242, 39292, 869, 7051, 25752, 248, 4006, 4156, 5568, 92046, 59220, 7256, 1381, 606, 1891, 93982, 3839, 67799, 2806, 1942, 3289, 64010, 159061, 27036, 2521, 2818, 3040, 141816, 152970, 6185, 8798, 2574, 82767, 1040, 7438, 1500, 5625, 1937, 327, 51562, 1980, 1365, 173307, 6948, 788, 3786, 55729, 8581, 6581, 2212, 53138, 1777, 6342, 70305, 150993, 82684, 2180, 99437, 116138, 1322, 274, 46850, 979, 46335, 4081, 1862, 1417, 121372, 5456, 114762, 43912, 2698, 67734, 8331, 47099, 3406, 137218, 3771, 138208, 140850, 7319, 79428, 6616, 98239, 4663, 55272, 3926, 6881, 8644, 190207, 72701, 7832, 15944

[8894, 1489, 171749, 595, 2370, 140110, 2059, 55729, 2350, 121372, 406, 2170, 6669, 3690, 6927, 7311, 543, 2046, 4522, 171701, 66198, 5205, 8753, 4441, 3451, 7482, 141816, 4074, 41617, 131578, 2818, 4156, 405, 2886, 147282, 4509, 370, 71341, 6342, 780, 98160, 118270, 2071, 73804, 2725, 30822, 67799, 2420, 3839, 5155, 3017, 39292, 50601, 134184, 3598, 182749, 8405, 112818, 7831, 3639, 32799, 4243, 2100, 3801, 8132, 7988, 47810, 188, 3165, 606, 4573, 43912, 152105, 95313, 5093, 1893, 26237, 3134, 106144, 51884, 173209, 46347, 4006, 6141, 46335, 6557, 99437, 3768, 3327, 92008, 56801, 64501, 7071, 83349, 31309, 6654, 31737, 1976, 4773, 5127, 149566, 4863, 101360, 44761, 5538, 4483, 111146, 61406, 88267, 542, 49132, 27704, 26375, 31702, 26631, 4052, 76738, 3551, 26357, 5636, 119218, 193, 99636, 6537, 2988, 55272, 85394, 280, 2108, 3211, 2360, 4115, 7345, 1752, 46578, 5710, 156781, 161830, 142509, 73015, 3899, 5502, 89386, 2534, 79677, 53447, 175485, 47382, 48161, 55555, 3524, 104218, 2698, 

[41573, 145, 49274, 3756, 4239, 3994, 139855, 189381, 144352, 62437, 70565, 193, 46347, 6306, 175485, 5384, 4573, 4180, 26524, 126088, 150254, 3165, 32914, 26375, 1862, 6669, 7938, 4630, 26357, 1107, 790, 5568, 101360, 112421, 97230, 337, 757, 5237, 175431, 1704, 2295, 8605, 107338, 3557, 121372, 5127, 3286, 44397, 46578, 117133, 6808, 53972, 2921, 95, 5538, 1021, 25752, 139747, 30822, 2116, 2212, 53447, 5364, 2692, 1417, 5752, 3042, 41566, 91842, 98122, 3524, 94494, 3714, 3981, 122916, 4785, 6285, 3451, 72701, 57368, 6305, 69844, 3912, 1184, 67295, 140301, 3497, 842, 3359, 67618, 156675, 159161, 164540, 69436, 2110, 6192, 4663, 4787, 207, 2046, 4629, 91353, 4156, 1414, 126548, 6058, 6644, 42556, 125916, 3932, 170401, 158872, 5455, 5562, 2504, 26590, 2313, 2108, 95135, 6837, 8753, 27416, 4967, 5502, 2043, 49132, 2698, 100527, 146730, 3839, 3341, 2170, 3000, 2932, 50601, 3040, 6616, 7064, 61406, 5799, 95510, 4509, 2068, 2725, 5489, 185473, 4558, 33171, 1365, 63276, 6538, 5074, 2947, 699

...

[2725, 4953, 3711, 8025, 81819, 3219, 2857, 3608, 31702, 6688, 132454, 5039, 4483, 140850, 6957, 4785, 2504, 5212, 3557, 1040, 74275, 3751, 3053, 179953, 4488, 4156, 32294, 426, 52579, 48262, 2690, 101025, 67799, 43460, 2212, 151315, 1496, 33896, 65738, 175485, 44195, 7179, 1996, 5041, 25752, 83480, 1707, 116817, 3359, 6390, 6557, 3286, 104837, 31309, 60408, 2878, 5585, 3327, 2082, 2100, 4081, 31367, 160978, 32657, 2932, 6185, 139747, 141890, 170401, 3932, 47810, 2682, 48738, 8656, 156675, 280, 2163, 44399, 5962, 2059, 33162, 53127, 3786, 164655, 2952, 27480, 55555, 5668, 78103, 64501, 54910, 40819, 33154, 2313, 93242, 2311, 88267, 3536, 5538, 79428, 175431, 56775, 8366, 8238, 73323, 67618, 6549, 4715, 41566, 2190, 5450, 116138, 2396, 2088, 60471, 3768, 7835, 6301, 101360, 173209, 70946, 7282, 183301, 6881, 69436, 337, 7019, 122892, 1172, 3313, 6424, 2713, 837, 69844, 115713, 6722, 7225, 4115, 145418, 50, 164540, 140301, 2886, 26265, 173941, 95313, 93988, 586, 68536, 49132, 2373, 5103,

[41617, 87444, 171749, 80124, 3219, 7282, 2514, 6945, 53972, 100527, 6713, 70361, 6245, 136359, 2314, 3557, 3313, 33779, 107338, 50356, 4139, 138208, 71304, 105250, 1185, 140289, 5954, 104760, 3507, 189381, 152079, 8331, 2737, 219, 4552, 279, 3213, 147282, 1783, 81782, 4509, 5364, 99, 98122, 96726, 4031, 3614, 3006, 2504, 60684, 6669, 6644, 8644, 51884, 280, 27036, 6187, 64501, 131578, 2671, 31223, 59604, 146730, 207, 57368, 70946, 668, 87869, 3786, 6502, 55444, 173209, 66090, 4395, 8238, 1439, 2065, 104837, 1033, 143525, 1996, 7757, 460, 95873, 152105, 26590, 104218, 37444, 102716, 77191, 5562, 70545, 2690, 59220, 1465, 2350, 289, 188, 54910, 32799, 6301, 1225, 7438, 5607, 5673, 81819, 55729, 2370, 25959, 3067, 6424, 5568, 4890, 5633, 152970, 6755, 543, 3957, 6223, 3756, 3635, 159161, 6305, 426, 27685, 56339, 2790, 74275, 33124, 132362, 89898, 168252, 2725, 27416, 7040, 55250, 66783, 8405, 159061, 47518, 6751, 3705, 4294, 6688, 1704, 7225, 2357, 232, 6100, 7451, 4987, 55555, 212, 1171

[5562, 7451, 5788, 69640, 99437, 164540, 88267, 173209, 147286, 33896, 126142, 3912, 4139, 4552, 50356, 4789, 31553, 130840, 64114, 89386, 6245, 627, 44397, 87869, 5585, 4616, 71619, 3614, 3771, 5041, 112818, 80586, 979, 3507, 4863, 6502, 930, 1489, 65130, 47382, 6041, 2989, 6755, 1417, 147282, 135137, 32666, 6713, 55444, 108795, 4164, 135861, 4395, 8894, 47774, 3640, 6881, 3926, 5489, 42761, 2682, 5040, 31367, 26622, 70697, 112421, 636, 7150, 114184, 3068, 2521, 5093, 6751, 40966, 26242, 93982, 49274, 99145, 5456, 7019, 2818, 48982, 85316, 50601, 82684, 54768, 1955, 477, 34153, 175485, 46347, 4204, 32460, 4896, 2212, 750, 8340, 828, 110553, 126548, 2534, 842, 66783, 1777, 112552, 1704, 27416, 8506, 55555, 147384, 145935, 33162, 5974, 5710, 156675, 55272, 7646, 3932, 257, 6654, 1033, 3635, 71899, 1465, 55729, 91353, 132454, 4509, 2313, 95135, 5607, 73804, 3213, 6994, 67799, 266, 3189, 31867, 6549, 2815, 44399, 27772, 5752, 67734, 2737, 80489, 68650, 4488, 136359, 2892, 4987, 138208, 64

[62437, 6216, 1417, 3823, 140301, 133780, 1040, 89386, 1914, 47382, 145, 7451, 337, 2212, 159441, 26357, 46578, 4135, 69640, 180031, 175387, 121372, 126548, 143365, 130452, 110553, 34520, 3327, 2145, 8331, 173209, 68886, 128914, 757, 2699, 31867, 26590, 3981, 63808, 3302, 1760, 5538, 6973, 60684, 48342, 5450, 64114, 5093, 3341, 5364, 51088, 142509, 78, 66090, 542, 4732, 2514, 3564, 1891, 7150, 3159, 114184, 2818, 6193, 136359, 2116, 3768, 285, 5733, 5585, 257, 112818, 42556, 190213, 27036, 7951, 4663, 93242, 48161, 95165, 4204, 2311, 146730, 115713, 2000, 3756, 4009, 140110, 165103, 2713, 156675, 3219, 902, 98239, 1955, 3342, 3671, 1012, 56775, 160978, 4505, 4987, 27731, 2180, 84952, 3042, 1942, 5612, 87960, 47774, 66783, 93831, 26313, 6502, 64575, 48774, 6305, 7646, 69746, 114707, 6306, 1151, 3908, 3690, 91353, 32666, 143525, 26631, 33154, 4042, 101360, 5455, 4947, 1996, 6348, 70565, 139747, 2190, 4, 4846, 67799, 40966, 95135, 126088, 3957, 6581, 70946, 3423, 54910, 4927, 42728, 11853

KeyboardInterrupt: 

**Task 7\*.** Implement an SVRRecommender - one-hot encode genres and fit an SVR model to 

(genre_1, genre_2, ..., genre_N) -> rating

Tune params of the SVR model to obtain as good results as you can. 

To do tuning properly (although in practive people are often happy with leave-one-out and do not bother with dividing the set into training, validation and test sets):
    - divide the set into training, validation and test sets (randomly divide the dataset in proportions 60%-20%-20%),
    - train the model with different sets of tunable parameters on the training set, 
    - choose the best tunable params based on results on the validation set, 
    - provide the final evaluation metrics on the test set for the best model obtained during tuning.

Recommended method of tuning: use hyperopt. Install the package using the following command: `pip install hyperopt`
    
Print the RMSE and MAE on the test set generated with numpy with seed 6789.

In [20]:
from sklearn.svm import SVR
from sklearn.preprocessing import MultiLabelBinarizer

class SVRRecommender(object):
    
    def __init__(self):
        self.model = None
        self.mlb = None
    
    def fit(self, interactions_df, users_df, items_df): 
        print('in fit')
        interactions_df = pd.merge(interactions_df, items_df, on='item_id')
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("-", "_", regex=False)
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(" ", "_", regex=False)
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.lower()
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.split("|")
        
        self.mlb = MultiLabelBinarizer()
        interactions_df = interactions_df.join(
            pd.DataFrame(self.mlb.fit_transform(interactions_df.pop('genres')),
                         columns=self.mlb.classes_,
                         index=interactions_df.index))
        
        x = interactions_df.loc[:, self.mlb.classes_].values
        y = interactions_df['rating'].values
    
        self.model = SVR(kernel='rbf', C=1.0, epsilon=0.1, gamma='scale').fit(x, y)
        print('end fit')
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        
        items_df = items_df.copy()
        items_df.loc[:, 'genres'] = items_df['genres'].str.replace("-", "_", regex=False)
        items_df.loc[:, 'genres'] = items_df['genres'].str.replace(" ", "_", regex=False)
        items_df.loc[:, 'genres'] = items_df['genres'].str.lower()
        items_df.loc[:, 'genres'] = items_df['genres'].str.split("|")
        
        items_df = items_df.join(
            pd.DataFrame(self.mlb.transform(items_df.pop('genres')),
                         columns=self.mlb.classes_,
                         index=items_df.index))
    
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            score = self.model.predict(items_df.loc[:, self.mlb.classes_].values)[0]
                
            user_recommendations = pd.DataFrame({'user_id': [user['user_id']],
                                                 'item_id': items_df.iloc[0]['item_id'],
                                                 'score': score})

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations

In [ ]:
svr_recommender = SVRRecommender()

results = [['SVRRecommender'] + list(evaluate_train_test_split_explicit(
    svr_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, seed=6789))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE'])

display(HTML(results.to_html()))